## 시작하기 전에

CIFAR-10 pretrained weight 받아오기

1. https://github.com/huyvnphan/PyTorch_CIFAR10 중간의 구글 드라이브 링크에서 zip 파일을 다운 (약 1기가)
2. 압축 해제 후 state_dicts 폴더를 구글 드라이브에 저장


CIFAR-10 pretrained model 받아오기
1. 아래 코드 실행


몇 가지 오류를 수정한 PytorchFI 라이브러리 받아오기
1. 아래아래 코드 실행

In [209]:
!git clone https://github.com/WaiNaat/PyTorch_CIFAR10.git

fatal: destination path 'PyTorch_CIFAR10' already exists and is not an empty directory.


In [210]:
!git clone https://github.com/WaiNaat/pytorchfi.git

fatal: destination path 'pytorchfi' already exists and is not an empty directory.


In [211]:
!pip install bitstring

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [212]:
import torch
import torchvision
import random
import copy
import numpy as np
import logging

from torchvision import transforms
from tqdm import tqdm

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [213]:
import pytorchfi
from pytorchfi.core import FaultInjection
import pytorchfi.weight_error_models as weight_error_models
from pytorchfi.util import random_value

# @INPROCEEDINGS{PytorchFIMahmoudAggarwalDSML20, author={A. {Mahmoud} and N. {Aggarwal} and A. {Nobbe} and J. R. S. {Vicarte} and S. V. {Adve} and C. W. {Fletcher} and I. {Frosio} and S. K. S. {Hari}}, booktitle={2020 50th Annual IEEE/IFIP International Conference on Dependable Systems and Networks Workshops (DSN-W)}, title={PyTorchFI: A Runtime Perturbation Tool for DNNs}, year={2020}, pages={25-31}, }

from PyTorch_CIFAR10.cifar10_models.vgg import vgg11_bn, vgg13_bn, vgg16_bn, vgg19_bn
from PyTorch_CIFAR10.cifar10_models.resnet import resnet18, resnet34, resnet50
from PyTorch_CIFAR10.cifar10_models.densenet import densenet121, densenet161, densenet169
from PyTorch_CIFAR10.cifar10_models.mobilenetv2 import mobilenet_v2
from PyTorch_CIFAR10.cifar10_models.googlenet import googlenet
from PyTorch_CIFAR10.cifar10_models.inception import inception_v3

## 설정 및 모델 불러오기

---

`model_name`, `model`: 위 셀의 `PyTorch_CIFAR10.cifar10_models` 에서 `import` 한 것들 중 하나      
`layer_type`: `['all']` 또는 `torch.nn.Modules`를 상속하는 클래스명으로 구성된 iterable   
`layer_nums`: `['all']` 또는 0 이상의 정수로 구성된 배열    
`corrupt_input_images`: `True`로 설정 시 inference 전 입력 이미지 자체에도 single bit flip 적용


In [214]:
logfile = logging.FileHandler(filename="/content/drive/MyDrive/소종/TempLog.log")
logger = logging.getLogger("TempLogger")
logger.addHandler(logfile)
logger.removeHandler(logging.StreamHandler())
print(logger.hasHandlers())
# 실험 환경 설정
model_name = "resnet18"
model = resnet18()
save_dir = 'resnet18.txt'

seed = 1234

batch_size = 256
img_size = 32
channels = 3

use_gpu = torch.cuda.is_available()

corrupt_input_images = True
quant_bits = 32
layer_type = ['all']
layer_nums = ['all']

True


In [215]:
logger.info("Start Log")

In [216]:
random.seed(seed)
torch.manual_seed(seed)

In [217]:
class add_input_layer(torch.nn.Module):

    def __init__(self, model, *args):
        super().__init__(*args)
        self.input_layer = torch.nn.Identity()
        self.model = model

    def forward(self, x):
        input = self.input_layer(x)
        output = self.model(input)
        return output

# single_bit_flip for weight 커스텀 모델
가중치에 bit-flip 삽입, 콘솔 창 출력 관련 변경 예정

*   get_quantized_data: 데이터 양자화
*   bit_flip_is_valid: soft error 모델링에 적합한 bit flip이 일어났는가?



In [218]:
# Custom Fault Injection Model
class single_bit_flip_func_weight(FaultInjection):
  def __init__(self, model, batch_size, input_shape, layer_types, **kwargs):
    super().__init__(self, model, batch_size, input_shape, layer_types, **kwargs)
    flip_bit = BitArray(hex='0x0000', length=32)
  def single_bit_flip_signed_for_weight(self, module, input_data, output_data):
    # OK...
    pass
  def get_bit_expression(self, data):
    # OK...
    pass
  def bit_flip_is_valid(self, original_data, corrupted_data):
    # this function will check if the bit flip is matching real world case
    # both original_data and corrupted_data are BitArray objects
    # ex: 0->1 (o), 1->0 (x)
    original_string = str(original_data)
    corrupted_string = str(corrupted_data)

In [219]:
# 모델 설정
path = f"/content/drive/My Drive/소종/2학기/state_dicts/{model_name}.pt"
model.load_state_dict(torch.load(path))

if corrupt_input_images:
    model = add_input_layer(model)

if use_gpu: model.to(device='cuda')

print(model)

add_input_layer(
  (input_layer): Identity()
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (outrelu): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      

## 데이터 전처리

In [220]:
# Normalization statics from https://github.com/huyvnphan/PyTorch_CIFAR10/blob/master/data.py
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize([0.4914, 0.4822, 0.4465], (0.2471, 0.2435, 0.2616))
    ]
)

data = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
dataset = torch.utils.data.DataLoader(data, batch_size=batch_size, shuffle=False, num_workers=0, drop_last=True)

Files already downloaded and verified


## Main

In [221]:
# single bit flip을 일으킬 모델 만들기
base_fi_model = FaultInjection(
    model = copy.deepcopy(model),
    batch_size = batch_size, 
    input_shape = [channels, img_size, img_size], 
    use_gpu = use_gpu,
#    bits = quant_bits,
    layer_types = layer_type
)
print(base_fi_model.print_pytorchfi_layer_summary())

============================ PYTORCHFI INIT SUMMARY ==============================

Layer types allowing injections:
----------------------------------------------------------------------------------
   - all

Model Info:
----------------------------------------------------------------------------------
   - Shape of input into the model: (3 32 32 )
   - Batch Size: 256
   - CUDA Enabled: True

Layer Info:
----------------------------------------------------------------------------------
Layer #       Layer type  Dimensions         Weight Shape         Output Shape
----------------------------------------------------------------------------------
    0         Identity           4       ['No weights']       [1, 3, 32, 32]
    1           Conv2d           4        [64, 3, 3, 3]      [1, 64, 32, 32]
    2      BatchNorm2d           4                 [64]      [1, 64, 32, 32]
    3             ReLU           4       ['No weights']      [1, 64, 32, 32]
    4        MaxPool2d           4   

In [222]:
# single bit flip을 수행할 layer 번호 정리
if 'all' in layer_nums:
    layer_nums = range(base_fi_model.get_total_layers())
else:
    layer_nums.sort()
    while layer_nums and layer_nums[-1] >= base_fi_model.get_total_layers():
        layer_nums.pop()

In [223]:
import random
con_out_array = []
def random_weight_location_mod(pfi, layer: int = -1):
    if layer == -1:
        layer = random.randint(0, pfi.get_total_layers() - 1)

    dim = pfi.get_weights_dim(layer)
    shape = pfi.get_weights_size(layer)

    dim0_shape = shape[0]
    if type(shape[0]) == str:
      print("Error shape:", shape[0])
      raise ValueError
    k = random.randint(0, dim0_shape - 1)
    if dim > 1:
        dim1_shape = shape[1]
        dim1_rand = random.randint(0, dim1_shape - 1)
    if dim > 2:
        dim2_shape = shape[2]
        dim2_rand = random.randint(0, dim2_shape - 1)
    else:
        dim2_rand = None
    if dim > 3:
        dim3_shape = shape[3]
        dim3_rand = random.randint(0, dim3_shape - 1)
    else:
        dim3_rand = None

    return ([layer], [k], [dim1_rand], [dim2_rand], [dim3_rand])
# 실험 진행
results = []

for layer_num in layer_nums:
  con_out_array.append(f"processing layer # {layer_num}")
  try:
    orig_correct_cnt = 0
    orig_corrupt_diff_cnt = 0

    for images, labels in dataset:

        if use_gpu:
            images = images.to(device='cuda')

        # 원본에 inference 진행
        model.eval()
        with torch.no_grad():
            orig_output = model(images)
        
        # single bit flip 위치 지정
        layer_num_list = []
        dim1 = []
        dim2 = []
        dim3 = []
        k_list = []
        value_list = []
        try:
          for batchNum in range(batch_size):
              try:
                layer, k, C, H, W = random_weight_location_mod(base_fi_model, layer=layer_num)
              except:
                raise ValueError
              value = random_value(-1,1)

              layer_num_list.append(layer)
              k_list.append(k)
              dim1.append(C)
              dim2.append(H)
              dim3.append(W)
              value_list.append(value)
              con_out_array.append(f"k value is {k} in batch # {batchNum} when position is {W}, {H}, {C}")
        except:
          con_out_array.append(f"Cannot check k value in layer {layer_num}")
          raise ValueError
        # corrupted model 만들기
        corrupted_model = base_fi_model.declare_weight_fault_injection(
            batch = [i for i in range(batch_size)],
            layer_num = layer_num_list,
            k = k_list,
            dim1 = dim1,
            dim2 = dim2,
            dim3 = dim3,
            value = value_list
        )
        
        # corrupted model에 inference 진행
        corrupted_model.eval()
        with torch.no_grad():
            corrupted_output = corrupted_model(images)
        
        # 결과 정리
        original_output = torch.argmax(orig_output, dim=1).cpu().numpy()
        corrupted_output = torch.argmax(corrupted_output, dim=1).cpu().numpy()
        labels = labels.numpy()

        # 결과 비교: 원본이 정답을 맞춘 경우 중 망가진 모델이 틀린 경우를 셈
        for i in range(batch_size):

            if labels[i] == original_output[i]:
                orig_correct_cnt += 1

                if original_output[i] != corrupted_output[i]:
                    orig_corrupt_diff_cnt += 1
        
    # 결과 저장
    result = f'Layer #{layer_num}: {orig_corrupt_diff_cnt} / {orig_correct_cnt} = {orig_corrupt_diff_cnt / orig_correct_cnt * 100:.4f}%'
    results.append(result)
  except:
    continue

Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights
Error shape: No weights


## 결과 파일 저장

In [224]:
logger.removeHandler(logfile)
logfile.close()

In [225]:
for result in results:
    print(result)

In [226]:
f = open('/content/drive/MyDrive/' + save_dir, 'w')

f.write(base_fi_model.print_pytorchfi_layer_summary())
f.write(f'\n\n===== Result =====\nSeed: {seed}\n')
for result in con_out_array:
    f.write(result + "\n")

f.close()